In [14]:
import pandas as pd
import pickle
from BayDS.lib.training import *
from sklearn.model_selection import GroupKFold, KFold

In [2]:
data = pd.read_pickle("temp/data.pkl")

In [3]:
data

,userid,field0,field1,field2,field3,field4,field5,field6,field7,field8,...,field27_count_groupby_field29,userid_count_groupby_field29,field5_count_groupby_userid,field7_count_groupby_userid,field8_count_groupby_userid,field10_count_groupby_userid,field12_count_groupby_userid,field26_count_groupby_userid,field27_count_groupby_userid,field29_count_groupby_userid
orderid,,,,,,,,,,,,,,,,,,,,,
1000000,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,1,...,154699,1,1,1,1,1,1,1,1,1
1000001,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,1,...,83311,1,1,2,2,2,1,1,1,1
1000002,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,1,...,2162,1,1,2,2,1,1,2,2,1
1000003,0392247b4b87674aba2c32bf2292b105771a6a376871be...,0,-0.238651,10,11,1,1,3,2,1,...,154699,1,1,1,3,2,1,2,2,1
1000004,d1aeefef311bbeb4bd84876c8d49421f276674527d5578...,0,-0.704079,8,11,1,1,0,1,1,...,199779,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455006,e54d8c57b0d2b64d19c59d5068372758067921f06101e2...,0,2.553918,8,8,1,1,10,2,0,...,199779,1,1,1,1,1,1,1,1,1
455007,5af2c8b53a965915f496971f0f105393f7f9c9b0931e6b...,0,-0.238651,9,9,1,1,1,1,1,...,199779,1,1,1,1,1,1,1,1,1
455008,255bfc6ac2f9c98d7f86e59bd72a682e7a50c2db610749...,0,-0.161080,10,11,1,1,0,1,1,...,154699,1,1,1,1,1,1,1,1,1


In [4]:
exclude_features_from_training = ['indicator_goal21', 'indicator_goal22', 'indicator_goal23',
       'indicator_goal24', 'indicator_goal25', 'goal21', 'goal22', 'goal23',
       'goal24', 'goal25', 'goal1','userid',]

In [6]:
X = data[data['goal1']>=0].drop(['goal1'], axis=1)
y = data[data['goal1']>=0]['goal1']
test=data[data['goal1']<0]
train_features = [f for f in data.columns if f not in exclude_features_from_training]

In [12]:
results = {}

In [18]:
NFOLDS = 5
folds = GroupKFold(n_splits=NFOLDS)

for random_state in range(1,10):
    params = {
              'objective': 'binary',
              "metric": 'auc',
              "verbosity": -1,
              'random_state':random_state
             }
    train_options = {
            "model_type":'lgb',
            "params": params,
            "eval_metric":'auc',
            'early_stopping_rounds': 100,
            'n_estimators': 500,
            'averaging': 'rank',
            'use_groups': False,
            'fold_name': folds.__class__.__name__,
            'n_splits': NFOLDS,

        }


    result_dict = train_model_classification_vb( X=X, 
                                                 X_test=test, 
                                                 columns=train_features,
                                                 y=y, 
                                                 params=params, folds=folds,
                                                 model_type=train_options['model_type'], 
                                                 plot_feature_importance=False,
                                                 verbose=100, early_stopping_rounds=train_options['early_stopping_rounds'],
                                                 n_estimators=train_options['n_estimators'], 
                                                 averaging=train_options['averaging'],
                                                 groups=X['userid'],
                                                 n_jobs=-1)
    results[random_state] = result_dict
    pickle.dump(results, open('temp/random.pkl','wb'))

Fold 1 started at Tue Dec 17 16:02:12 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.907298	training's auc: 0.907298	valid_1's auc: 0.672828	valid_1's auc: 0.672828
Early stopping, best iteration is:
[56]	training's auc: 0.858881	training's auc: 0.858881	valid_1's auc: 0.676408	valid_1's auc: 0.676408
Fold 2 started at Tue Dec 17 16:03:12 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.903609	training's auc: 0.903609	valid_1's auc: 0.713336	valid_1's auc: 0.713336
Early stopping, best iteration is:
[32]	training's auc: 0.804053	training's auc: 0.804052	valid_1's auc: 0.72533	valid_1's auc: 0.72533
Fold 3 started at Tue Dec 17 16:04:08 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.898954	training's auc: 0.898954	valid_1's auc: 0.68824	valid_1's auc: 0.68824
Early stopping, best iteration is:
[36]	training's auc: 0.819776	training's auc: 0.819776	valid_1's a

[100]	training's auc: 0.902784	training's auc: 0.902784	valid_1's auc: 0.667931	valid_1's auc: 0.667931
Early stopping, best iteration is:
[34]	training's auc: 0.815296	training's auc: 0.815296	valid_1's auc: 0.68055	valid_1's auc: 0.68055
Fold 5 started at Tue Dec 17 16:23:34 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.905042	training's auc: 0.905042	valid_1's auc: 0.672842	valid_1's auc: 0.672842
Early stopping, best iteration is:
[32]	training's auc: 0.809654	training's auc: 0.809653	valid_1's auc: 0.683304	valid_1's auc: 0.683305
CV mean score: 0.6933, std: 0.0181.
Fold 1 started at Tue Dec 17 16:24:34 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.907298	training's auc: 0.907298	valid_1's auc: 0.672828	valid_1's auc: 0.672828
Early stopping, best iteration is:
[56]	training's auc: 0.858881	training's auc: 0.858881	valid_1's auc: 0.676408	valid_1's auc: 0.676408
Fold 2 started at Tue Dec 17 1

In [21]:
for k,v in results.items():
    print (k, v['scores'])

0 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
1 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
2 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
3 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
4 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
5 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
6 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
7 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
8 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
9 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.68054955